## Importando as bibliotecas necessárias

In [2]:
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, cohen_kappa_score, confusion_matrix
from sklearn.neural_network import MLPClassifier
from skimage.io import imread, imshow
import numpy as np
from glob import glob
from tqdm.notebook import tqdm
import pandas as pd
import pickle

## Carregando o modelo da FaceNet

O modelo utilizado neste trabalho foi adquirido através deste repositório: https://github.com/nyoki-mtl/keras-facenet.

In [2]:
model = keras.models.load_model('./facenet_keras.h5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [3]:
model.summary()

_________________
Block8_1_Branch_1_Conv2d_0b_1x3 (None, 3, 3, 192)    0           Block8_1_Branch_1_Conv2d_0b_1x3_B
__________________________________________________________________________________________________
Block8_1_Branch_0_Conv2d_1x1 (C (None, 3, 3, 192)    344064      Mixed_7a[0][0]                   
__________________________________________________________________________________________________
Block8_1_Branch_1_Conv2d_0c_3x1 (None, 3, 3, 192)    110592      Block8_1_Branch_1_Conv2d_0b_1x3_A
__________________________________________________________________________________________________
Block8_1_Branch_0_Conv2d_1x1_Ba (None, 3, 3, 192)    576         Block8_1_Branch_0_Conv2d_1x1[0][0
__________________________________________________________________________________________________
Block8_1_Branch_1_Conv2d_0c_3x1 (None, 3, 3, 192)    576         Block8_1_Branch_1_Conv2d_0c_3x1[0
___________________________________________________________________________________________

## Descompactando a base de imagens

In [4]:
!unzip -q './new_dataset.zip'

## Extraindo os embeddings das imagens da base

In [ ]:
maskon = glob('./new_dataset/maskon/*.png')
maskoff = glob('./new_dataset/maskoff/*.png')

len(maskon), len(maskoff)

(1087, 1000)

In [ ]:
data = maskon + maskoff

len(data)

2087

In [ ]:
embeddings = []

for path in tqdm(data):

  try:
    # Lendo e normalizando a imagem
    img = imread(path).astype('float32')/255
    
    # Aplicando um reshape na imagem para deixar o formato de acordo com o input da FaceNet
    input = np.expand_dims(img, axis=0)

    # Extraindo o vetor de embeddings
    embeddings.append(model.predict(input)[0])

  except:
    print(f'Error in {path}')
    continue

In [ ]:
np.array(embeddings).shape

(2087, 128)

In [ ]:
pd.DataFrame(embeddings).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127
0,0.529861,-0.116981,-1.665346,1.248521,1.598799,0.278045,0.304103,-0.118939,1.774264,0.826556,0.077844,-0.306842,-1.549261,0.167605,-1.212324,-0.480293,-0.261717,-0.925149,-1.935170,0.501111,0.898265,-0.273530,0.146744,-0.005715,1.168411,0.118356,0.167026,-0.047558,-0.811278,-0.237024,1.317334,-0.390440,-0.673231,0.260476,-0.343329,0.595729,-0.035986,-1.252293,0.426928,-0.367467,...,1.402963,0.303098,0.635723,0.164443,0.382001,1.354455,1.130473,0.636062,-0.524057,-0.715174,-1.203014,0.694434,0.634969,1.196817,-0.682478,-2.487039,0.603308,-1.276406,-0.160437,-0.328320,0.563077,-0.339303,0.781790,-0.374037,-0.007167,0.385545,0.238083,2.293101,-1.002626,1.867401,0.385438,0.249096,0.009145,0.588885,0.711551,-1.256024,0.881116,-2.619466,0.337068,-0.658998
1,-1.465220,-0.168793,-2.338724,-0.668507,0.965428,-0.208923,0.046012,0.839133,0.644278,0.223989,-0.456473,-0.259854,-0.144440,0.663159,1.199390,-0.464565,-0.985120,0.410346,-0.918254,-0.694171,-0.170305,-0.377948,-0.440538,-0.877064,0.032340,-0.404194,1.278263,-1.495868,-1.954007,0.220253,0.200204,-0.155451,0.562642,-0.042883,0.149769,-0.083641,1.047029,-0.657857,1.585847,-1.314621,...,-1.541661,-0.697578,0.527608,0.960666,-0.200147,0.713711,-0.439225,0.960253,0.493430,0.445061,-0.503433,0.879527,0.101686,0.232556,-0.170840,-0.072412,-0.105298,0.222335,-1.017816,-0.198275,-0.176340,0.777148,0.175407,-2.124459,0.095475,0.149310,-0.174501,1.357557,-0.678132,1.425699,0.262793,0.872560,-0.090062,1.013145,0.978253,-0.424967,-1.179652,-1.910363,0.224178,-0.229611
2,0.512470,-0.866971,0.390168,0.780297,0.849814,0.761520,1.074221,0.135706,0.199493,0.722926,-0.944703,-0.246431,1.613663,0.622039,0.133762,0.775980,-1.258771,0.405735,0.850086,-0.055061,-0.189246,0.430769,2.205356,0.352916,0.163450,1.010188,0.073460,0.086306,0.037241,0.106199,0.705615,-1.828221,0.494748,0.094524,1.542593,-0.617536,-1.839192,0.149347,0.996711,0.971689,...,0.971803,-0.940096,-0.162577,1.376448,-0.062721,1.217687,-0.181922,0.647990,-1.233003,-0.054968,-0.810201,-0.882251,0.999346,0.494710,-0.003434,-0.877864,0.072594,-0.372083,0.010628,-0.340639,-0.278274,-0.951026,0.779574,1.060986,-0.487930,-0.920609,1.196717,1.667161,0.661297,0.359585,-0.068681,-0.635781,0.047876,0.698600,-0.175519,0.160645,1.101048,-0.895452,-0.365417,-1.053375
3,0.198832,-1.645907,-0.731764,-0.494559,-1.268282,0.494078,0.296694,-0.116334,1.342171,-0.478131,0.203683,-0.238544,-0.481152,0.143304,-1.518094,-0.038010,0.414376,-0.419738,0.137051,0.111295,0.275177,-1.081456,1.498750,-1.322842,0.518105,1.060379,0.213990,-1.105707,0.363045,0.377797,0.160829,0.247080,-0.323462,1.014648,0.095680,0.559495,-0.219360,0.223671,2.137155,-0.763412,...,-0.529661,0.084290,-0.162958,-0.354054,-1.205797,0.125379,-0.032348,1.266086,-1.542529,-1.066011,-0.701763,-0.003786,0.491211,1.393355,0.543272,-0.852652,-0.288690,1.144328,0.050054,-1.371761,-0.584191,-1.276465,1.032401,0.171432,0.082084,-0.651352,1.055359,0.799398,-0.027075,-0.227083,-0.312684,-0.142750,-1.230165,0.243662,1.430314,-0.409941,-0.617603,-0.282606,0.291231,-0.698395
4,-0.407711,0.072958,0.135241,-1.383544,-0.183764,-0.707488,-0.778080,1.961451,0.892836,-0.953716,1.302660,0.712708,-0.932156,0.034510,-0.743828,-0.583242,1.583809,-1.107608,0.000909,-0.243127,-0.860185,-0.175993,-0.950702,0.011450,-0.833621,0.370138,1.872560,-1.117092,-2.016923,-1.296110,0.346373,0.533437,0.202883,1.877035,0.275119,0.959660,0.968927,-1.332040,1.528625,0.133902,...,-0.975839,0.641004,-1.396660,0.356531,-1.106392,-0.570447,1.276065,1.006763,-0.509495,0.081475,-0.614136,1.005900,-0.153889,-0.193652,0.082957,0.350123,-0.375246,0.904999,-0.044433,0.019714,-1.958796,-1.574999,1.332880,-1.954255,1.061178,0.186442,-0.406546,1.673800,-0.430974,0.334766,2.496941,-0.

Adicionando labels no dataframe de embeddings

* Com máscara: label **1**
* Sem máscara: label **0**

In [ ]:
labels = pd.DataFrame({
    'label': [1]*len(maskon) + [0]*len(maskoff)
})

df_embeddings = pd.concat([pd.DataFrame(embeddings), labels], axis=1)
df_embeddings

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,label
0,0.529861,-0.116981,-1.665346,1.248521,1.598799,0.278045,0.304103,-0.118939,1.774264,0.826556,0.077844,-0.306842,-1.549261,0.167605,-1.212324,-0.480293,-0.261717,-0.925149,-1.935170,0.501111,0.898265,-0.273530,0.146744,-0.005715,1.168411,0.118356,0.167026,-0.047558,-0.811278,-0.237024,1.317334,-0.390440,-0.673231,0.260476,-0.343329,0.595729,-0.035986,-1.252293,0.426928,-0.367467,...,0.303098,0.635723,0.164443,0.382001,1.354455,1.130473,0.636062,-0.524057,-0.715174,-1.203014,0.694434,0.634969,1.196817,-0.682478,-2.487039,0.603308,-1.276406,-0.160437,-0.328320,0.563077,-0.339303,0.781790,-0.374037,-0.007167,0.385545,0.238083,2.293101,-1.002626,1.867401,0.385438,0.249096,0.009145,0.588885,0.711551,-1.256024,0.881116,-2.619466,0.337068,-0.658998,1
1,-1.465220,-0.168793,-2.338724,-0.668507,0.965428,-0.208923,0.046012,0.839133,0.644278,0.223989,-0.456473,-0.259854,-0.144440,0.663159,1.199390,-0.464565,-0.985120,0.410346,-0.918254,-0.694171,-0.170305,-0.377948,-0.440538,-0.877064,0.032340,-0.404194,1.278263,-1.495868,-1.954007,0.220253,0.200204,-0.155451,0.562642,-0.042883,0.149769,-0.083641,1.047029,-0.657857,1.585847,-1.314621,...,-0.697578,0.527608,0.960666,-0.200147,0.713711,-0.439225,0.960253,0.493430,0.445061,-0.503433,0.879527,0.101686,0.232556,-0.170840,-0.072412,-0.105298,0.222335,-1.017816,-0.198275,-0.176340,0.777148,0.175407,-2.124459,0.095475,0.149310,-0.174501,1.357557,-0.678132,1.425699,0.262793,0.872560,-0.090062,1.013145,0.978253,-0.424967,-1.179652,-1.910363,0.224178,-0.229611,1
2,0.512470,-0.866971,0.390168,0.780297,0.849814,0.761520,1.074221,0.135706,0.199493,0.722926,-0.944703,-0.246431,1.613663,0.622039,0.133762,0.775980,-1.258771,0.405735,0.850086,-0.055061,-0.189246,0.430769,2.205356,0.352916,0.163450,1.010188,0.073460,0.086306,0.037241,0.106199,0.705615,-1.828221,0.494748,0.094524,1.542593,-0.617536,-1.839192,0.149347,0.996711,0.971689,...,-0.940096,-0.162577,1.376448,-0.062721,1.217687,-0.181922,0.647990,-1.233003,-0.054968,-0.810201,-0.882251,0.999346,0.494710,-0.003434,-0.877864,0.072594,-0.372083,0.010628,-0.340639,-0.278274,-0.951026,0.779574,1.060986,-0.487930,-0.920609,1.196717,1.667161,0.661297,0.359585,-0.068681,-0.635781,0.047876,0.698600,-0.175519,0.160645,1.101048,-0.895452,-0.365417,-1.053375,1
3,0.198832,-1.645907,-0.731764,-0.494559,-1.268282,0.494078,0.296694,-0.116334,1.342171,-0.478131,0.203683,-0.238544,-0.481152,0.143304,-1.518094,-0.038010,0.414376,-0.419738,0.137051,0.111295,0.275177,-1.081456,1.498750,-1.322842,0.518105,1.060379,0.213990,-1.105707,0.363045,0.377797,0.160829,0.247080,-0.323462,1.014648,0.095680,0.559495,-0.219360,0.223671,2.137155,-0.763412,...,0.084290,-0.162958,-0.354054,-1.205797,0.125379,-0.032348,1.266086,-1.542529,-1.066011,-0.701763,-0.003786,0.491211,1.393355,0.543272,-0.852652,-0.288690,1.144328,0.050054,-1.371761,-0.584191,-1.276465,1.032401,0.171432,0.082084,-0.651352,1.055359,0.799398,-0.027075,-0.227083,-0.312684,-0.142750,-1.230165,0.243662,1.430314,-0.409941,-0.617603,-0.282606,0.291231,-0.698395,1
4,-0.407711,0.072958,0.135241,-1.383544,-0.183764,-0.707488,-0.778080,1.961451,0.892836,-0.953716,1.302660,0.712708,-0.932156,0.034510,-0.743828,-0.583242,1.583809,-1.107608,0.000909,-0.243127,-0.860185,-0.175993,-0.950702,0.011450,-0.833621,0.370138,1.872560,-1.117092,-2.016923,-1.296110,0.346373,0.533437,0.202883,1.877035,0.275119,0.959660,0.968927,-1.332040,1.528625,0.133902,...,0.641004,-1.396660,0.356531,-1.106392,-0.570447,1.276065,1.006763,-0.509495,0.081475,-0.614136,1.005900,-0.153889,-0.193652,0.082957,0.350123,-0.375246,0.904999,-0.044433,0.019714,-1.958796,-1.574999,1.332880,-1.954255,1.061178,0.186442,-0.406546,1.673800,-0.430974,0.334766,2.496941,-0.889891,-2.097792,-0.797771,-0.628648,

## Dividindo os dados em treino e teste

In [ ]:
X = np.array(df_embeddings.drop('label', axis=1))
y = np.array(df_embeddings['label'])

X.shape, y.shape

((2087, 128), (2087,))

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((1669, 128), (418, 128), (1669,), (418,))

## Treinando uma multilayer perceptron (MLP) para a classificação dos embeddings

Instanciando o modelo

In [ ]:
mlp_model = MLPClassifier()

Treinando o modelo

In [ ]:
mlp_model.fit(X_train, y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

Avaliando o modelo com base no conjunto de teste

In [ ]:
y_pred = mlp_model.predict(X_test)

In [ ]:
print('Acurácia: ', accuracy_score(y_test, y_pred))
print('Kappa: ', cohen_kappa_score(y_test, y_pred))
print('Matriz de confusão:\n', confusion_matrix(y_test, y_pred))

Acurácia:  0.9569377990430622
Kappa:  0.9135907389117303
Matriz de confusão:
 [[187  14]
 [  4 213]]


## Treinando MLP novamente, mas agora com todos os dados

Neste momento, já avaliamos nosso classificador com base no conjunto de teste e chegamos ao melhor cenário. Agora precisamos treinar esse modelo novamente, mas dessa vez usando todas as imagens para treino. Quanto mais dados inserirmos em nosso modelo, melhor será seu aprendizado.

Além disso, nesta etapa também precisamos salvar o modelo, pois será ele que usaremos no algoritmo de detecção com a OpenCV e a MTCNN.

In [ ]:
X.shape, y.shape

((2087, 128), (2087,))

In [ ]:
mlp_model = MLPClassifier()

In [ ]:
mlp_model.fit(X, y)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=None, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [ ]:
mlp_model.score(X, y)

0.9995208433157643

Salvando o modelo como um arquivo pickle

In [ ]:
pickle.dump(mlp_model, open('./mlp_model.pkl', 'wb'))